In [233]:
# all necessary imports
import json
import csv
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import r2_score, mean_squared_error

In [234]:
# importing the dataset from json
df= pd.read_json('/Users/jyotit-kaushal/github/boozeless-analytics/data/singapore_geo_dataset.json')

In [235]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11865 entries, 0 to 11864
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   venue_id              11865 non-null  object 
 1   latitude              11865 non-null  float64
 2   longitude             11865 non-null  float64
 3   venue_name            11865 non-null  object 
 4   platform_listing_url  11865 non-null  object 
 5   region                11865 non-null  object 
 6   neighborhood          11865 non-null  object 
 7   address               11826 non-null  object 
 8   phone                 8416 non-null   object 
 9   website               7481 non-null   object 
 10  price_point_bucket    4372 non-null   object 
 11  review_count          11227 non-null  float64
 12  average_rating        11227 non-null  float64
 13  review_sample         11865 non-null  object 
 14  opening_hours         11865 non-null  object 
 15  busy_times            11

In [236]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11865 entries, 0 to 11864
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   venue_id              11865 non-null  object 
 1   latitude              11865 non-null  float64
 2   longitude             11865 non-null  float64
 3   venue_name            11865 non-null  object 
 4   platform_listing_url  11865 non-null  object 
 5   region                11865 non-null  object 
 6   neighborhood          11865 non-null  object 
 7   address               11826 non-null  object 
 8   phone                 8416 non-null   object 
 9   website               7481 non-null   object 
 10  price_point_bucket    4372 non-null   object 
 11  review_count          11227 non-null  float64
 12  average_rating        11227 non-null  float64
 13  review_sample         11865 non-null  object 
 14  opening_hours         11865 non-null  object 
 15  busy_times            11

In [237]:
# creating a ranking system governed by rating and number of reviews using bayesian probability
df_temp1= df[df['review_count']>=100]
r= df_temp1['average_rating'].mean()
w= (df['review_count'].mean())/5

bayesian_weighted_rating= ((r*w)+(df['average_rating']*df['review_count']))/(w+df['review_count'])
df['bayesian_weighted_rating']= bayesian_weighted_rating



In [238]:
categorical_columns= ['region', 'neighborhood', 'price_point_bucket', 'platform_category', 'venue_segment']


df_temp2= df.copy()

encoder = LabelEncoder()
for col in categorical_columns:
    df_temp2[col] = encoder.fit_transform(df_temp2[col])

train_data = df_temp2.dropna(subset=['bayesian_weighted_rating'])
test_data = df_temp2[df_temp2['bayesian_weighted_rating'].isna()]

X_train = train_data[categorical_columns]
y_train = train_data['bayesian_weighted_rating']
X_test = test_data[categorical_columns]

model = RandomForestRegressor()
model.fit(X_train, y_train)

pred_y_train= model.predict(X_train)
r2= r2_score(y_train, pred_y_train)

print("R2 Score:", r2)

mse = mean_squared_error(y_train, pred_y_train)
print("Mean Squared Error:", mse)

missing_review_predictions = model.predict(X_test)

df.loc[df['bayesian_weighted_rating'].isna(), 'bayesian_weighted_rating'] = missing_review_predictions


R2 Score: 0.3983521408283993
Mean Squared Error: 0.0383236445140175


In [239]:

busy_during_nighttime = []

for busy_time_dict in df['busy_times']:
    if busy_time_dict:
        times=[]
        for day, set_times in busy_time_dict.items():
            midnight= list(busy_time_dict[day].values())[:1]
            nighttime= list(busy_time_dict[day].values())[-7:]
            times.extend(midnight)
            times.extend(nighttime)

        if((sum(times)/(8*6))>40):
            busy_during_nighttime.append("Yes")
        else:
            busy_during_nighttime.append("No")
    else:
        busy_during_nighttime.append(None)
            
print(busy_during_nighttime)

df['busy_during_nighttime']=busy_during_nighttime

['NA', 'NA', 'NA', 'NA', 'Yes', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'No', 'NA', 'NA', 'Yes', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'No', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'No', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'Yes', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 

In [240]:
df.shape

(11865, 23)

In [241]:
df_temp3 = df.copy()

atmosphere = []
crowd = []
dining_options = []
happy_hour = []
highlights = []
offerings = []
payments = []
planning = []
types_of_alcohol = []
amenities_cat = []
accessibility = []
ordering_options = []

for amenities in df['venue_amenities']:
    if amenities:
        if 'amenities' in amenities:
            amenities_cat.append(amenities['amenities'])
        else:
            amenities_cat.append(None)

        if 'atmosphere' in amenities:
            atmosphere.append(amenities['atmosphere'])
        else:
            atmosphere.append(None)
        
        if 'crowd' in amenities:
            crowd.append(amenities['crowd'])
        else:
            crowd.append(None)
            
        if 'dining_options' in amenities:
            dining_options.append(amenities['dining_options'])
        else:
            dining_options.append(None)

        if 'ordering_options' in amenities:
            ordering_options.append(amenities['ordering_options'])
        else:
            ordering_options.append(None)
            
        if 'happy_hour' in amenities:
            happy_hour.append(amenities['happy_hour'])
        else:
            happy_hour.append(None)
            
        if 'highlights' in amenities:
            highlights.append(amenities['highlights'])
        else:
            highlights.append(None)
            
        if 'offerings' in amenities:
            offerings.append(amenities['offerings'])
        else:
            offerings.append(None)
            
        if 'payments' in amenities:
            payments.append(amenities['payments'])
        else:
            payments.append(None)
            
        if 'planning' in amenities:
            planning.append(amenities['planning'])
        else:
            planning.append(None)
            
        if 'types_of_alcohol' in amenities:
            types_of_alcohol.append(amenities['types_of_alcohol'])
        else:
            types_of_alcohol.append(None)

        if 'accessability' in amenities:
            accessibility.append(amenities['accessability'])
        else:
            accessibility.append(None)

    else:
        atmosphere.append(None)
        crowd.append(None)
        dining_options.append(None)
        happy_hour.append(None)
        highlights.append(None)
        offerings.append(None)
        payments.append(None)
        planning.append(None)
        types_of_alcohol.append(None)
        amenities_cat.append(None)
        ordering_options.append(None)
        accessibility.append(None)

df['atmosphere'] = atmosphere
df['crowd'] = crowd
df['dining_options'] = dining_options
df['happy_hour'] = happy_hour
df['highlights'] = highlights
df['offerings'] = offerings
df['payments'] = payments
df['planning'] = planning
df['types_of_alcohol'] = types_of_alcohol
df['ordering_options'] = ordering_options
df['amenities_cat'] = amenities_cat
df['accessibility'] = accessibility




In [242]:
df = df.drop(columns=['average_rating', 'busy_times', 'venue_amenities', 'venue_amenities'])

In [243]:
categorical_columns= ['region', 'neighborhood', 'platform_category', 'venue_segment', 'bayesian_weighted_rating']


df_temp4= df.copy()

encoder = LabelEncoder()
for col in categorical_columns:
    df_temp4[col] = encoder.fit_transform(df_temp4[col])

df_temp4['price_point_bucket']= encoder.fit_transform(df_temp4['price_point_bucket'])

df_temp4.loc[df_temp4['price_point_bucket'] == 4, 'price_point_bucket'] = None


train_data = df_temp4.dropna(subset=['price_point_bucket'])
test_data = df_temp4[df_temp4['price_point_bucket'].isna()]

X_train = train_data[categorical_columns]
y_train = train_data['price_point_bucket']
X_test = test_data[categorical_columns]

model = RandomForestClassifier()
model.fit(X_train, y_train)

pred_y_train= model.predict(X_train)
r2= r2_score(y_train, pred_y_train)

print("R2 Score:", r2)

mse = mean_squared_error(y_train, pred_y_train)
print("Mean Squared Error:", mse)

missing_pricepoint_predictions = model.predict(X_test)


def change_to_symbols(lst):
    symbols = {0: '$', 1: '$$', 2: '$$$', 3: '$$$$'}
    result = []

    for value in lst:
        if value in symbols:
            result.append(symbols[value])
        else:
            result.append(value)

    return result

missing_pricepoint_predictions= change_to_symbols(missing_pricepoint_predictions)

df.loc[df['price_point_bucket'].isna(), 'price_point_bucket'] = missing_pricepoint_predictions

R2 Score: 0.9972061423458972
Mean Squared Error: 0.0011436413540713633


In [244]:
df['review_sample']= df['review_sample'].astype(str)

In [245]:
df.shape

(11865, 32)

In [246]:
pivot_table_nightbusy = pd.pivot_table(df, 
                            index='busy_during_nighttime', 
                            aggfunc='size', 
                            fill_value=0)

print(pivot_table_nightbusy)

busy_during_nighttime
NA     11208
No       395
Yes      262
dtype: int64


In [247]:
# pivot table for price_point_bucket
pivot_table_pricepoint = pd.pivot_table(df, 
                            index='price_point_bucket', 
                            aggfunc='size', 
                            fill_value=0)

print(pivot_table_pricepoint)

price_point_bucket
$       3629
$$      7473
$$$      649
$$$$     114
dtype: int64


In [248]:
# pivot table for venue_segment
pivot_table_venueseg = pd.pivot_table(df, 
                            index='venue_segment', 
                            aggfunc='size', 
                            fill_value=0)

print(pivot_table_venueseg)

venue_segment
Bar               816
Cafe             1520
Entertainment     341
Hawker Food      2326
Hotel             474
Non Alcoholic     532
Restaurant       5545
Stores            302
dtype: int64


In [249]:
pivot_table_venuesubseg = pd.pivot_table(df, 
                            index='venue_subsegment', 
                            aggfunc='size', 
                            fill_value=0)

print(pivot_table_venuesubseg)

venue_subsegment
Alcohol           287
Asian            2344
Bar               816
Cafe             1520
Entertainment     341
Fast Food         248
Fine Dining        24
Halal             171
Hawker Food      2326
Hotel             474
Non Alcoholic     532
Restaurant       2128
Stores             15
Vegetarian        130
Western           500
dtype: int64


In [ ]:
# pivot table for reviews

df['review_count_10000'] = df['review_count'].round(-3)

pivot_table_reviewcnt = pd.pivot_table(df, 
                            index='review_count_10000', 
                            aggfunc='size', 
                            fill_value=0)

print(pivot_table_reviewcnt)

review_count_10000
0.0        9335
1000.0     1420
2000.0      250
3000.0       99
4000.0       46
5000.0       25
6000.0       22
7000.0        8
8000.0        5
9000.0        3
10000.0       2
11000.0       1
12000.0       2
13000.0       2
15000.0       1
17000.0       2
21000.0       1
24000.0       1
43000.0       1
83000.0       1
dtype: int64


In [ ]:
df['review_sample_size'] = df['review_sample'].apply(lambda x: len(x))

pivot_table_reviewsample = pd.pivot_table(df, 
                            index='review_sample_size', 
                            aggfunc='size', 
                            fill_value=0)

print(pivot_table_reviewsample)

review_sample_size
0    10522
1       78
2       43
3       27
4       24
5       14
6       24
7        6
8     1127
dtype: int64


In [ ]:
pd.set_option('display.max_rows', None)

pivot_table_platformcat = pd.pivot_table(df, 
                            index='platform_category', 
                            aggfunc='size', 
                            fill_value=0)

print(pivot_table_platformcat)

platform_category
American restaurant                       21
Argentinian restaurant                     2
Art cafe                                   6
Asian fusion restaurant                   22
Asian restaurant                          66
Authentic Japanese restaurant             16
Bakery                                   532
Bangladeshi restaurant                     9
Bar                                      532
Bar & grill                               46
Barbecue restaurant                       14
Beer garden                               15
Beer store                                12
Bistro                                   147
Breakfast restaurant                       2
Brewery                                   12
Brewpub                                    5
Brunch restaurant                          3
Buffet restaurant                         15
Cafe                                    1509
Cafeteria                                  2
Cantonese restaurant                 

In [ ]:
print(df.iloc[40]['review_sample'])

count=0
for i in range(0, len(df)):
    if len(df.iloc[i]['review_sample'])==0:
        pass
    else:
        # print(df.iloc[i]['review_sample'])
        count+=1

print(count)

{}
1343
